## DNNs on Loihi

This notebook provides a minimal example of a keyword spotter trained offline and then run on Loihi. To keep things short for this tutorial, we'll implement a single layer of 200 neurons on the chip, and use this network to predict character transcriptions of audio signals. First we'll import some utility functions for initializing weights and for converting a nengo simulation output to text characters.

The purpose of this example illustrative - since we're keeping things short and simple, we're not aiming to get a high-accuracy model; rather, we're trying to summarize the key steps in the process of building such a model.

In [1]:
import pickle
import nengo
import nengo_dl
import nengo_loihi

import tensorflow as tf
import numpy as np

from utils import predict_text, ce_loss, weight_init, create_stream

import warnings
warnings.filterwarnings('ignore')

/Users/peterblouw/anaconda/envs/iceland/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## 1. Initial Setup

First, we set some high level parametes and load the data we'll be training the model on.

In [2]:
inp_dim = 390
out_dim = 29

n_neurons = 200
max_rate = 250
amplitude = 1 / max_rate

lifs = nengo.LIF(tau_rc=0.02, tau_ref=0.002, amplitude=amplitude)

with open('./data/data.pkl', 'rb') as pfile:
    train_data = pickle.load(pfile)

with open('./data/stream.pkl', 'rb') as pfile:
    inference_data = pickle.load(pfile)

## 2. Define the Network

This is very similar to how you typically use Nengo: create collections of neurons, then connect them to input nodes to provide data to the network. We'll just add a couple of flags to configure how the network is handled by Nengo DL:

In [3]:
with nengo.Network() as net:
    net.config[nengo.Connection].synapse = None
    net.config[nengo.Ensemble].max_rates = nengo.dists.Choice([max_rate])
    net.config[nengo.Ensemble].intercepts = nengo.dists.Choice([0])

    inp = nengo.Node(np.zeros(inp_dim))
    ens = nengo.Ensemble(n_neurons=n_neurons, dimensions=1, neuron_type=lifs)
    out = nengo.Node(size_in=out_dim)

    conn_a = nengo.Connection(
        inp, ens.neurons, transform=weight_init(shape=(n_neurons, inp_dim)))
    
    conn_b = nengo.Connection(
        ens.neurons, out, transform=weight_init(shape=(out_dim, n_neurons)))

    probe = nengo.Probe(out)

## 3. Train the Network

Now we can take our constructed model and use Nengo DL to optimize its parameters with some minimal tweaks:

In [4]:
# create a Nengo DL simulator and set the minibatch size
with nengo_dl.Simulator(net, minibatch_size=100) as sim:

    # define an optimizer
    optimizer = tf.train.RMSPropOptimizer(0.001)
    
    # specify inputs and target
    inputs = {inp: train_data['inp']}
    targets = {probe: train_data['out']}
    
    # define a loss function
    objective = {probe: ce_loss}

    # optimize the model parameters
    sim.train(
        inputs, targets, optimizer, n_epochs=35, objective=objective)
    
    # collect the parameters to port to loihi
    params = sim.get_nengo_params([ens, conn_a, conn_b])

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Training finished in 0:00:18 (loss: 8.1746)                                    


## 4. Rebuild the Network for Loihi

Now, we reconstruct the network, initializing the ensemble and connections with the trained parameters.

In [5]:
with nengo.Network() as loihi_net:
    loihi_net.config[nengo.Connection].synapse = None

    inp = nengo.Node(np.zeros(inp_dim))
    ens = nengo.Ensemble(n_neurons=n_neurons, dimensions=1, neuron_type=lifs, **params[0])
    out = nengo.Node(size_in=out_dim)

    conn_a = nengo.Connection(
        inp, ens.neurons, transform=params[1]['transform'])

    conn_b = nengo.Connection(
        ens.neurons, out, transform=params[2]['transform'])

    probe = nengo.Probe(out, synapse=0.01)
    loihi_net.inp = inp

## 5. Run the Keyword Spotter on Loihi

Here we'll use an emulator for the chip. Once the inputs are setup, it's just one line of code to run the keyword spotter on Loihi!

In [6]:
for features, text in inference_data[:10]:
    n_steps = features.shape[0]
    loihi_net.inp.output = create_stream(features)

    sim = nengo_loihi.Simulator(loihi_net, target='sim')

    with sim:
        sim.run_steps(n_steps)
        prediction = predict_text(sim, probe, n_steps)
        print('Correct Text: %s' % text)
        print('Predicted Text: %s' % prediction)
        print('')

Correct Text: aloha
Predicted Text: aloha

Correct Text: aloha
Predicted Text: aloha

Correct Text: aloha
Predicted Text: aloha

Correct Text: aloha
Predicted Text: aloha

Correct Text: aloha
Predicted Text: aloha

Correct Text: aloha
Predicted Text: aloha

Correct Text: aloha
Predicted Text: aloh

Correct Text: aloha
Predicted Text: aloa

Correct Text: aloha
Predicted Text: aloha

Correct Text: aloha
Predicted Text: aloha



Though this model is not particularly performant, we can build more sophisticated models that perform comparably to a standard DNN implementation in Tensorflow!